## Scenario Visualization
The purpose of this notebook is to visualize the projects in a scenario that you constructed.

In [1]:
import os
import sys
import pandas as pd
import network_wrangler as wr

from ipywidgets import Output, HTML, HBox, VBox, Layout, AppLayout
from IPython.display import display,IFrame

%config IPCompleter.greedy=True
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 1000)

import logging
logger = logging.getLogger("WranglerLogger")
logger.handlers[0].stream = sys.stdout
# if you don't want to see so much detail, set to logging.INFO or DEBUG
logger.setLevel(logging.INFO)

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
#if you need to reset variables in the notebook
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? n
Nothing done.


### Define Base Files, Networks, and Scenario

In [3]:
STPAUL_DIR = os.path.join(os.getcwd(),'../','examples','stpaul')
STPAUL_SHAPE_FILE = os.path.join(STPAUL_DIR,"shape.geojson")
STPAUL_LINK_FILE = os.path.join(STPAUL_DIR,"link.json")
STPAUL_NODE_FILE = os.path.join(STPAUL_DIR,"node.geojson")

In [4]:
road_net = wr.RoadwayNetwork.read(
    link_file= STPAUL_LINK_FILE, 
    node_file=STPAUL_NODE_FILE, 
    shape_file=STPAUL_SHAPE_FILE, 
    fast=True,
    shape_foreign_key ='shape_id',
)

2020-11-13 16:20:57, INFO: Reading RoadwayNetwork
2020-11-13 16:20:57, INFO: Reading RoadwayNetwork
2020-11-13 16:21:03, INFO: Read 66253 links from /Users/elizabeth/Documents/urbanlabs/MetCouncil/working/network_wrangler/notebook/../examples/stpaul/link.json
2020-11-13 16:21:03, INFO: Read 66253 links from /Users/elizabeth/Documents/urbanlabs/MetCouncil/working/network_wrangler/notebook/../examples/stpaul/link.json
2020-11-13 16:21:03, INFO: Read 17159 nodes from /Users/elizabeth/Documents/urbanlabs/MetCouncil/working/network_wrangler/notebook/../examples/stpaul/node.geojson
2020-11-13 16:21:03, INFO: Read 17159 nodes from /Users/elizabeth/Documents/urbanlabs/MetCouncil/working/network_wrangler/notebook/../examples/stpaul/node.geojson
2020-11-13 16:21:03, INFO: Read 66253 shapes from /Users/elizabeth/Documents/urbanlabs/MetCouncil/working/network_wrangler/notebook/../examples/stpaul/shape.geojson
2020-11-13 16:21:03, INFO: Read 66253 shapes from /Users/elizabeth/Documents/urbanlabs/Me

In [5]:
transit_net = wr.TransitNetwork.read(STPAUL_DIR)

2020-11-13 16:21:17, INFO: Read in transit feed from: /Users/elizabeth/Documents/urbanlabs/MetCouncil/working/network_wrangler/notebook/../examples/stpaul
2020-11-13 16:21:17, INFO: Read in transit feed from: /Users/elizabeth/Documents/urbanlabs/MetCouncil/working/network_wrangler/notebook/../examples/stpaul
2020-11-13 16:21:17, INFO: Removing calendar.txt from transit network config because file not found
2020-11-13 16:21:17, INFO: Removing calendar.txt from transit network config because file not found
2020-11-13 16:21:18, INFO: Removing calendar_dates.txt from transit network config because file not found
2020-11-13 16:21:18, INFO: Removing calendar_dates.txt from transit network config because file not found
2020-11-13 16:21:18, INFO: Removing fare_attributes.txt from transit network config because file not found
2020-11-13 16:21:18, INFO: Removing fare_attributes.txt from transit network config because file not found
2020-11-13 16:21:18, INFO: Removing fare_rules.txt from transit 

In [6]:
base_scenario = {
    "road_net": road_net,
    "transit_net": transit_net,
}

### Build a Scenario by Specifying Specific Project Cards

In [7]:
BUILD_CARD_FILENAMES = [
    '3_multiple_roadway_attribute_change.yml',
    'multiple_changes.yml',
    '4_simple_managed_lane.yml',
]

project_cards_list = [
    wr.ProjectCard.read(os.path.join(STPAUL_DIR, "project_cards", filename), validate=False)
    for filename in BUILD_CARD_FILENAMES
]

In [10]:
my_scenario = wr.Scenario.create_scenario(
    base_scenario=base_scenario, 
    project_cards_list=project_cards_list
)
my_scenario.apply_all_projects()

2020-11-13 16:24:46, INFO: Creating Scenario
2020-11-13 16:24:46, INFO: Creating Scenario
2020-11-13 16:24:46, INFO: Applying test managed lane project
2020-11-13 16:24:46, INFO: Applying test managed lane project
2020-11-13 16:24:46, INFO: Applying Project to Roadway Network: test managed lane project
2020-11-13 16:24:46, INFO: Applying Project to Roadway Network: test managed lane project
2020-11-13 16:24:46, INFO: Applying road diet on one way couplet 5th and 6th
2020-11-13 16:24:46, INFO: Applying road diet on one way couplet 5th and 6th
2020-11-13 16:24:46, INFO: Applying road diet on one way couplet 5th and 6th
2020-11-13 16:24:46, INFO: Applying road diet on one way couplet 5th and 6th
2020-11-13 16:24:46, INFO: Applying Project to Roadway Network: road diet on one way couplet 5th and 6th
2020-11-13 16:24:46, INFO: Applying Project to Roadway Network: road diet on one way couplet 5th and 6th
2020-11-13 16:24:47, INFO: Applying road diet on one way couplet 5th and 6th
2020-11-13 

In [11]:
my_scenario.applied_projects

['test managed lane project',
 'road diet on one way couplet 5th and 6th',
 '6th Street Transitway']

## Project Summaries

The method `Scenario.applied_project_card_summary()` 

In [12]:
scenario_summary = my_scenario.scenario_summary()

2020-11-13 16:24:53, INFO: Summarizing Scenario
2020-11-13 16:24:53, INFO: Summarizing Scenario


In [13]:
with out:
    display(HTML(scenario_summary))


NameError: name 'out' is not defined

In [14]:
out = Output(layout={'height':'auto','width':'98%','border': '4px solid grey','overflow_y': 'visible', "right-margin": '15px',"padding": '5px', 'object_fit': 'scale-down'})
out.clear_output(wait=True)

with out:
    for pc in my_scenario.project_cards:
        s = my_scenario.applied_project_card_summary(pc.__dict__)
        display(HTML("<h2>PROJECT CARD: "+pc.project+"</h2><hr>"))
        display(HTML("<var>Source File:</var><code>"+pc.file+"</code>"))
        display(HTML("<b>Changes in project: "+str(s['total_parts'])+"</b>"))
        for p in range(s['total_parts']):
            part = "Part "+str(p+1)
            part_summary = s[part]
            display(HTML("<h3>"+part_summary['project']+"</h3></br><var>Change Category:</var>"+part_summary['category']))
            display(part_summary)
            display(part_summary['map'])

In [15]:
out

Output(layout=Layout(border='4px solid grey', height='auto', object_fit='scale-down', overflow_y='visible', pa…